<a href="https://colab.research.google.com/github/matheus896/crewai-examples/blob/main/Crew_Raspagem_Web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --q crewai
!pip install --q litellm
!pip install --q 'crewai[tools]'

In [4]:
import os
from crewai_tools import ScrapeWebsiteTool
from crewai import Agent, Task, Crew, Process, LLM
from litellm import completion

# NOSSAS MENTES

In [5]:
os.environ["GEMINI_API_KEY"] = "SUA_API_KEY"

llm = LLM (
    model = 'gemini/gemini-1.5-flash',
    temperature=0.2,
    api_key="GEMINI_API_KEY"
    )

# RASPANDO A WEB

In [6]:
ainew = ScrapeWebsiteTool(
    website_url="https://www.artificialintelligence-news.com/"
)

In [7]:
forbes = ScrapeWebsiteTool(
    website_url="https://www.forbes.com/ai/"
)


# Agentes

In [8]:
planejador = Agent(
    role="Planejador de postagem",
    goal="Planejar conteúdo envolvente para instagram sobre {topic}",
    backstory="Você está trabalhando no planejamento de {n} posts para o instagram "
              "sobre o tema: {topic}. "
              "Você coleta informações que ajudam o "
              "público se informar sobre {topic}. "
              "Seu trabalho é a base para que "
              "o Pesquisador de Conteúdo procure na web sobre {topic}.",
    verbose=True,
    llm=llm,
    allow_delegation=False,
    memory=True
)


In [9]:
pesquisador = Agent(
    role='Pesquisador',
    goal='Pesquisar tendências para postagens sobre {topic} na área '
         'de tecnologia com base no planejamento do Planejador. '
         'Seu trabalho é a base para que '
         'o escritor possa escrever {n} posts sobre {topic}',
    verbose=True,
    backstory="Você é um pesquisador experiente, sempre em busca das últimas tendências e informações relevantes sobre {topic}.",
    llm=llm,
    tools=[ainew, forbes],
    allow_delegation=False,
    max_retry_limit = 2,
    max_execution_time = 60,
    memory=True
)

In [10]:
escritor = Agent(
    role='Escritor',
    goal='Escrever {n} postagens cativantes em português do Brasil para o Instagram sobre {topic} com no mínimo 250 palavras e no máximo 350 palavras.'
         'Seu trabalho é a base para que o fotografo possa escrever prompts de imagens para os {n} posts',
    backstory="Você é um escritor criativo, capaz de transformar informações em conteúdo atraente para postagens no Instagram.",
    llm=llm,
    verbose=True,
    allow_delegation=False,
    memory=True
)


In [12]:
gerente = Agent(
    role='Gerente de postagens',
    goal=("""Supervisione o trabalho de uma equipe de postagens no Instagram. Você é bem crítico em relação
            ao que vai ser postado no Instagram da empresa de notícias na área da tecnologia.
            Você delegará tarefas à sua equipe e fará perguntas esclarecedoras
            para revisar e aprovar as {n} posts sobre {topic} que foram solicitadas pela direção da empresa."""),
    backstory=("""Você é um gerente experiente, sempre em busca das últimas tendências e informações relevantes.
                 Você está trabalhando com uma nova demanda e faz com que sua equipe realize o trabalho da
                 melhor forma possível."""),
    llm=llm,
    verbose=True,
    memory=True

)


# Tarefas

In [13]:
plano_task = Task(
    description=(
        "1. Priorize as últimas tendências, principais 'players', "
            "e notícias relevantes sobre {topic}.\n"
        "2. Identifique o público-alvo, considerando "
            "seus interesses e pontos de dor.\n"
        "3. Desenvolva um plano de conteúdo detalhado, incluindo "
            "uma introdução, pontos principais e um chamado à ação.\n"
        "4. Inclua palavras-chave de SEO e dados ou fontes relevantes."
    ),
    expected_output="Um documento de plano de conteúdo para {n} posts sobre {topic} "
        "com um esboço, análise do público, "
        "palavras-chave de SEO e recursos.",
    agent=planejador,
    verbose=True
)

In [14]:
pesquisa_task = Task(
    description="Pesquise as últimas tendências sobre {topic}.",
    expected_output="Um relatório detalhado sobre as tendências mais recentes sobre {topic} na área de tecnologia.",
    agent=pesquisador,
    verbose=True,
    context=[plano_task]
)

In [16]:
escrita_task = Task(
    description=("""Escreva {n} postagens envolventes em português do Brasil
                        com base nas tendências pesquisadas sobre {topic} com no mínimo 250 palavras e no máximo 350 palavras cada.
                        Cada post deve ser formatado como:
                        \n\nPOST:\ntexto do post em português do brasil
                        \n\nPROMPT:\nPrompt da imagem desse post\n\n"""),
    expected_output="{n} postagens de Instagram bem escritas, atraentes e em português do Brasil, formatadas conforme especificado para o tópico {topic}.",
    agent=escritor,
    verbose=True,
    context=[plano_task, pesquisa_task]
)


In [17]:
revisao_task = Task(
description=("""Revise as {n} escritas para as {n} postagens do cliente e garanta
                          que as informações de cada postagem estejam organizadas, sem erros e cativantes
                          em português do Brasil sobre {topic}.
                          Certifique-se de que cada post está formatado como:
                          \n\nPOST:\ntexto do post em português do brasil
                          \n"""),
expected_output="{n} textos organizados por post, revisados e prontos para serem publicados em português do Brasil, formatados conforme especificado.",
agent=gerente,
verbose=True,
context=[plano_task, pesquisa_task, escrita_task]

)



# CRIANDO A TRIPULAÇÃO

In [18]:
crew = Crew(
    agents=[planejador, pesquisador, escritor, gerente],
    tasks=[plano_task, pesquisa_task, escrita_task, revisao_task],
    process=Process.sequential,
    verbose=True
)


In [ ]:
result = crew.kickoff(inputs={
      'topic':'Inteligencia Artificial',
      'n':1})

In [20]:
from IPython.display import Markdown
Markdown(result.raw)

## POST:

🤖 **A inteligência artificial está mudando o mundo, e você precisa entender porquê!** 🤖

A IA já está presente em diversas áreas da nossa vida, desde os aplicativos que usamos no celular até os carros autônomos. Mas o que realmente está por trás dessa revolução tecnológica? 🤔

🧠 **A IA aprende e evolui com base em dados, tornando-se cada vez mais inteligente e capaz de realizar tarefas complexas.** 

🚀 **E as possibilidades são infinitas!** 🚀

**Veja alguns exemplos:**

* **IA Generativa:** Criando imagens, textos e músicas incríveis com ferramentas como ChatGPT, DALL-E e Midjourney. 🎨
* **IA na Saúde:** Diagnósticos mais precisos, tratamentos personalizados e descobertas de novas curas. 🏥
* **IA na Indústria:** Automação de processos, otimização da produção e aumento da segurança. 🏭
* **IA na Educação:** Personalização do aprendizado, ferramentas de ensino inovadoras e acesso à educação de qualidade para todos. 📚

**O futuro da IA é promissor, mas também exige atenção.** ⚠️

É fundamental discutir os impactos da IA na sociedade, como a ética do uso de dados e a necessidade de garantir que a tecnologia seja utilizada para o bem de todos. 🤝

**Acompanhe nossas próximas postagens para saber mais sobre as tendências e os desafios da inteligência artificial!** 

#inteligenciaartificial #IA #futuro #tecnologia #inovacao #chatgpt #dalle #midjourney #saude #industria #educacao #transformacaodigital #oportunidades #desafios #futurodatecnologia #IAgenerativa #etica #dados #sociedade 


## PROMPT:

Uma imagem abstrata e futurista que represente a inteligência artificial. A imagem pode ser composta por linhas, formas geométricas, cores vibrantes e elementos que evoquem a ideia de algoritmos, dados, conexões e a constante evolução da IA. A imagem deve transmitir a sensação de dinamismo, complexidade e progresso.